In [52]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Ensure the environment variables are set
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')

if not langchain_api_key:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment variables.")
if not huggingface_api_key:
    raise ValueError("HUGGINGFACE_API_KEY is not set in the environment variables.")

# Set environment variables for the application
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
os.environ['HUGGINGFACE_API_KEY'] = huggingface_api_key

In [53]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [54]:
# Load Documents (use PyPDFLoader for PDF)
file_path = r"C:\Users\User\Desktop\NSU\CSE299 Materials\LLM\Dataset\Diabetes_Care_BADAS_guideline2019-3.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

docs[0].page_content[:1000]

'DIABETES CARE \nBADAS Guideline 2019 \n          \n  \n   \n  \n   P|) \nDAS GUELINE ON Man \nDELIT IGEMEN \n  \nA Joint Initiative of \nDiabetic Association of Bangladesh \nNCDC Program, Directorate General of Health Services'

In [55]:
# Main Chunking Functions
from chunking_evaluation.chunking import (
    ClusterSemanticChunker,
    LLMSemanticChunker,
    FixedTokenChunker,
    RecursiveTokenChunker,
    KamradtModifiedChunker
)
# Additional Dependencies
import tiktoken
from chromadb.utils import embedding_functions
import spacy
import os

# Helper Function for Analyzing Chunking

In [56]:
def analyze_chunks(chunks, use_tokens=False):
    # Print the chunks of interest
    print("\nNumber of Chunks:", len(chunks))
    print("\n", "="*50, "200th Chunk", "="*50,"\n", chunks[199])
    print("\n", "="*50, "201st Chunk", "="*50,"\n", chunks[200])
    
    chunk1, chunk2 = chunks[199], chunks[200]
    
    if use_tokens:
        encoding = tiktoken.get_encoding("cl100k_base")
        tokens1 = encoding.encode(chunk1)
        tokens2 = encoding.encode(chunk2)
        
        # Find overlapping tokens
        for i in range(len(tokens1), 0, -1):
            if tokens1[-i:] == tokens2[:i]:
                overlap = encoding.decode(tokens1[-i:])
                print("\n", "="*50, f"\nOverlapping text ({i} tokens):", overlap)
                return
        print("\nNo token overlap found")
    else:
        # Find overlapping characters
        for i in range(min(len(chunk1), len(chunk2)), 0, -1):
            if chunk1[-i:] == chunk2[:i]:
                print("\n", "="*50, f"\nOverlapping text ({i} chars):", chunk1[-i:])
                return
        print("\nNo character overlap found")

# Character Text Splitting

In [57]:
all_text = " ".join([doc.page_content for doc in docs])

def chunk_text(docs, chunk_size, overlap):
    chunks = []
    stride = chunk_size - overlap
    current_idx = 0
    
    while current_idx < len(all_text):
        # Take chunk_size characters starting from current_idx
        chunk = docs[current_idx:current_idx + chunk_size]
        if not chunk:  # Break if we're out of text
            break
        chunks.append(chunk)
        current_idx += stride  # Move forward by stride
    
    return chunks

In [58]:
character_overlap_chunks = chunk_text(all_text, chunk_size=800, overlap=400)

analyze_chunks(character_overlap_chunks)


Number of Chunks: 296

 ================================================== 200th Chunk ================================================== 
 t should be done annually with 75g 2h OGTT or 
A1C. If prediabetes, reassessment should be done 6 monthly and should be put 
on MNT or metformin should be followed by standard protocol. 
Low dose estrogen-progesterone can be offered for contraception. 
Progesterone only preparation increases risk of vascular complications. 
Screening for all components of metabolic syndrome should be offered. 
Throughout the period of breast feeding all types of insulins and metformin 
can be safely used in lactating women. Women with preprgnancy diabetes 
who are breastfeeding should continue to avoid any drugs for the treatment of 
diabetes complications that were discontinued for safety reasons in the 
pre-conception period. 
6.2 Diabetes in children 
Diabetes mellitus in childhood and adolescence is mo

 ================================================== 201st

# Token Text Splitting

In [59]:
import tiktoken

# Loading cl100k_base tokenizer 
encoder = tiktoken.get_encoding("cl100k_base")

# Text Example
text = "Diabetes"
tokens = encoder.encode(text)

print("Tokens:", tokens)

Tokens: [22427, 16629]


In [60]:
for i in range(len(tokens)):
    print(f"Token {i+1}:", encoder.decode([tokens[i]]))

print("Full Decoding: ", encoder.decode(tokens))

Token 1: Di
Token 2: abetes
Full Decoding:  Diabetes


# Helper Function for Counting Tokens

In [61]:
def count_tokens(text, model="cl100k_base"):
    """Count tokens in a text string using tiktoken"""
    encoder = tiktoken.get_encoding(model)
    return print(f"Number of tokens: {len(encoder.encode(text))}")

# Character Text Splitting

In [62]:
all_text = " ".join([doc.page_content for doc in docs])

def chunk_text(docs, chunk_size, overlap):
    chunks = []
    stride = chunk_size - overlap
    current_idx = 0
    
    while current_idx < len(all_text):
        # Take chunk_size characters starting from current_idx
        chunk = docs[current_idx:current_idx + chunk_size]
        if not chunk:  # Break if we're out of text
            break
        chunks.append(chunk)
        current_idx += stride  # Move forward by stride
    
    return chunks

In [63]:
character_overlap_chunks = chunk_text(all_text, chunk_size=800, overlap=400)

analyze_chunks(character_overlap_chunks)


Number of Chunks: 296

 ================================================== 200th Chunk ================================================== 
 t should be done annually with 75g 2h OGTT or 
A1C. If prediabetes, reassessment should be done 6 monthly and should be put 
on MNT or metformin should be followed by standard protocol. 
Low dose estrogen-progesterone can be offered for contraception. 
Progesterone only preparation increases risk of vascular complications. 
Screening for all components of metabolic syndrome should be offered. 
Throughout the period of breast feeding all types of insulins and metformin 
can be safely used in lactating women. Women with preprgnancy diabetes 
who are breastfeeding should continue to avoid any drugs for the treatment of 
diabetes complications that were discontinued for safety reasons in the 
pre-conception period. 
6.2 Diabetes in children 
Diabetes mellitus in childhood and adolescence is mo

 ================================================== 201st

# Token Text Splitting

In [64]:
import tiktoken

# Loading cl100k_base tokenizer 
encoder = tiktoken.get_encoding("cl100k_base")

# Text Example
text = "diabetes"
tokens = encoder.encode(text)

print("Tokens:", tokens)

Tokens: [8747, 16629]


In [65]:
for i in range(len(tokens)):
    print(f"Token {i+1}:", encoder.decode([tokens[i]]))

print("Full Decoding: ", encoder.decode(tokens))

Token 1: di
Token 2: abetes
Full Decoding:  diabetes


# Helper Function for Counting Tokens

In [66]:
def count_tokens(docs, model="cl100k_base"):
    """Count tokens in a text string using tiktoken"""
    encoder = tiktoken.get_encoding(model)
    return print(f"Number of tokens: {len(encoder.encode(all_text))}")

In [67]:
all_text = " ".join([doc.page_content for doc in docs])
    
fixed_token_chunker = FixedTokenChunker(
    chunk_size=150, 
    chunk_overlap=0,
    encoding_name="cl100k_base"
)

token_chunks = fixed_token_chunker.split_text(all_text)

analyze_chunks(token_chunks, use_tokens=True)



Number of Chunks: 203

 ================================================== 200th Chunk ================================================== 
  25.3 Kg/m2.He 
has presented with general weakness for last 2 months. There is nothing 
remarkable on physical examination. His OGTT was done 3 days back; report shows: 
FBG- 16.2 mmol/L; 2 hr post-glucose value- 24.1 mmol/L; HbA1c- 14.5%. There is 
nothing remarkable on other laboratory tests. What will be the management 
stratigy plan for him? 
  
76 ‘158 Diabetes Care: BADAS Guideline 2019 Abbreviation 
ACR 
AGI 
ALA 
ALT 
ASCVD 
BG 
BMI 
BP 
BUN 
CAD 
CGM 


 ================================================== 201st Chunk ================================================== 
 CHF 
CKD 
CVD 
eGFR 
EPA 
DHA 
DM 
ESRD 
GAD 
hCG 
HbAic 
HDL 
HNF 
IHD 
ISPAD 
LAD 
LDL 
MDI 
MNT 
MODY 
NAFLD 
NPH 
PCI 
PCOS 
RBC 
SMBG 
TG 
UTI 
Albumin-to-creatinine ratio 
Alpha-glucosidase inhibitors 
a-Linolenic acid 
Alanine aminotransferase 
Atherosclerotic cardio

In [68]:
count_tokens(token_chunks[0])

Number of tokens: 30370


In [69]:
fixed_token_chunker = FixedTokenChunker(
    chunk_size=150, 
    chunk_overlap=50,
    encoding_name="cl100k_base"
)

token_overlap_chunks = fixed_token_chunker.split_text(all_text)

analyze_chunks(token_overlap_chunks, use_tokens=True)


Number of Chunks: 304

 ================================================== 200th Chunk ================================================== 
 Plasma Glucose values Treatment Change of treatment Treatment 
at onset if target not reviewed 
achieved in & continued 
FPG 292 to 109 NPT 2 week/1 week NPT+PT 
mg/dl mg/dl Uncomplicated/ 
(25.1 (6.0 complicated 
mmol/L) mmol/L) 
and/or 
2hPPG 2120 to <140 NPT 2 week/1 week NPT+PT 
mg/dl mg/dl Uncomplicated/ 
(26.7 (7.8 complicated 
mmol/L) mmol/L) 
FPG >110 to <126 NPT 1 week NPT+PT 
mg/dl

 ================================================== 201st Chunk ================================================== 
 mg/dl mg/dl Uncomplicated/ 
(26.7 (7.8 complicated 
mmol/L) mmol/L) 
FPG >110 to <126 NPT 1 week NPT+PT 
mg/dl mg/dl 
(26.1 (7.0) 
mmol/L) mmol/L) 
and/or 
2hPPG 2140 to <200 NPT 1 week NPT+PT 
mg/dl mg/dl 
(27.8 (11.1 
mmol/L) mmol/L) 
FPG 2=126 mg/dl NPT+PT x NPT+PT 
(27.0 mmol/L) 
and

Overlapping text (50 tokens): mg/dl mg/dl Uncomplicated/

# Recursive Character Text Splitter

In [70]:
recursive_character_chunker = RecursiveTokenChunker(
    chunk_size=400,  # Character Length
    chunk_overlap=0,  # Overlap
    length_function=len,  # Character length with len()
    separators=["\n\n", "\n", ".", "?", "!", " ", ""] # According to Research
)

recursive_character_chunks = recursive_character_chunker.split_text(all_text)
analyze_chunks(recursive_character_chunks, use_tokens=False)


Number of Chunks: 322

 ================================================== 200th Chunk ================================================== 
 25-35 (11.5-16.0) 
15-25 (7.0-11.5) 
11-20 (5-9.0) 
No recommendation 
37-54 (16.8-24.5) 
31-50 (14.1-22.7) 
25-42 (11.4-19.1) 
  
52 IE Diabetes Care: BADAS Guideline 2019 Meal pattern 
Three meals and 3 snacks should be taken including one snack at bed time. This 
approach can be adopted where help of nutritionist service is not available. 
Recommended overall total caloric distribution

 ================================================== 201st Chunk ================================================== 
 Carbohydrate: 33-40% with low glycemic index 
Protein: ~ 20% 
Fat: < 40%, saturated fat <7% and transfat<1% 
Plate model can be practiced 
Simple sugars should be avoided. Food containing complex carbohydrate 
intake is recommended 
High dietary fibre and whole grain containing foods should be encouraged. 
Non-calorie sweeteners (aspartame) may be

In [71]:
len(recursive_character_chunks[199]) # Chunk 200

391

In [72]:
recursive_character_chunker = RecursiveTokenChunker(
    chunk_size=400,  # Character Length
    chunk_overlap=200,  # Overlap
    length_function=len,  # Character length with len()
    separators=["\n\n", "\n", ".", "?", "!", " ", ""] # According to Research
)

recursive_character_overlap_chunks = recursive_character_chunker.split_text(all_text)
analyze_chunks(recursive_character_overlap_chunks, use_tokens=False)


Number of Chunks: 595

 ================================================== 200th Chunk ================================================== 
 Empagliflozin 10 mg x 1,morning 25 mg (in 1 dose) eGFR <45- avoid 
eGFR <30- contraind 
Ertugliflozin 5 mg x 1,morning 15 mg (in 1 dose) eGFR <60-avoid 
eGFR <30-contraind         
NB. ac- before meal; pc- after meal. 
eGFR ml/min/1.73 m2. 
  
28 lll Diabetes Care: BADAS Guideline 2019 Classification of insulin 
Insulin type Onset TLC) PVE Li «:) 
  
Bolus (prandial) insulins

 ================================================== 201st Chunk ================================================== 
 NB. ac- before meal; pc- after meal. 
eGFR ml/min/1.73 m2. 
  
28 lll Diabetes Care: BADAS Guideline 2019 Classification of insulin 
Insulin type Onset TLC) PVE Li «:) 
  
Bolus (prandial) insulins 
Rapid-acting insulin analogues 
* Insulin Aspart 
* Insulin Glulisine 
* Insulin Lispro 
Short-acting (Regular) insulins 
Basal insulins 
Intermediate-acting (NPH)

# Recursive Token Text Splitter

In [73]:
# Count the number of tokens in each page_content
def token_count(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string, disallowed_special=()))
    return num_tokens

In [74]:
recursive_token_chunker = RecursiveTokenChunker(
    chunk_size=150,  # Character Length
    chunk_overlap=0,  # Overlap
    length_function=token_count,
    separators=["\n\n", "\n", ".", "?", "!", " ", ""] # According to Research
)

recursive_token_chunks = recursive_token_chunker.split_text(all_text)

analyze_chunks(recursive_token_chunks, use_tokens=True)


Number of Chunks: 235

 ================================================== 200th Chunk ================================================== 
 Pre-Ramadan dinner dose may be reduced by 20-50% 
for Suhoor (depending on prolonged fasting, carb 
content) and glycemic control 
  
68 II Diabetes Care: BADAS Guideline 2019 SMBG 
Good to do: Pre Suhoor — before taking meal, early morning- 2 hrs after Suhoor, 
around 10 am, mid-day - 11 am to 2 pm, pre Iftar meal, 2 hrs after Iftar and at any 
time when there are symptoms of hypoglycemia / hyperglycemia or feeling unwell.

 ================================================== 201st Chunk ================================================== 
 Must to do : Before Iftar, 2 hour after Iftar, Mid-day 
Frequency of SMBG should be daily for first 3 days, every 3rd day from next week 
onwards and every alternateday in the last week. 
6.7 Hajj and travel 
It could be estimated that Muslims with diabetes performing Hajj may exceed 
220000 per year. During Haj

In [75]:
recursive_token_chunker = RecursiveTokenChunker(
    chunk_size=150,  # Character Length
    chunk_overlap=50,  # Overlap
    length_function=token_count,
    separators=["\n\n", "\n", ".", "?", "!", " ", ""] # According to Research
)

recursive_token_overlap_chunks = recursive_token_chunker.split_text(all_text)

analyze_chunks(recursive_token_overlap_chunks, use_tokens=True)


Number of Chunks: 334

 ================================================== 200th Chunk ================================================== 
 presence of ASCVD risk. 
<40 Yes High intensity statin; if LDL =>70 mg/dl despite 
of therapy consider combining with ezetimibe. 
240 No Moderate intensity statin or high intensity 
statin may be considered based on risk-benefit 
profile or presence of ASCVD risk. 
240 Yes High intensity statin; if LDL >70 mg/dl despite 
of therapy, consider combining with ezetimibe. 
    
NB: Intensity of statin therapy may need to be adjusted according to individual 
patient response to medication (eg side effects, tolerability, LDL cholesterol 
levels, etc).

 ================================================== 201st Chunk ================================================== 
 NB: Intensity of statin therapy may need to be adjusted according to individual 
patient response to medication (eg side effects, tolerability, LDL cholesterol 
levels, etc). 
Statin therapy

# Semantic Chunker

In [80]:
# Helper Function from the Repo that Returns Embeddings
embedding_function = embedding_functions.HuggingFaceEmbeddingFunction(api_key=os.environ["HUGGINGFACE_API_KEY"], model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")

In [81]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

lc_semantic_chunker = SemanticChunker(HuggingFaceEmbeddings())

lc_semantic_chunks = lc_semantic_chunker.create_documents([all_text])

In [82]:
print("# of Chunks:", len(lc_semantic_chunks), "\n")
print(lc_semantic_chunks[19].page_content)
print("\n\n", "="*50, "\n\n")
print(lc_semantic_chunks[20].page_content)
print("\n\n", "="*50, "\n\n")

count_tokens(lc_semantic_chunks[19].page_content)
count_tokens(lc_semantic_chunks[20].page_content)

# of Chunks: 41 

eGFR ml/min/1.73 m2. 28 lll Diabetes Care: BADAS Guideline 2019 Classification of insulin 
Insulin type Onset TLC) PVE Li «:) 
  
Bolus (prandial) insulins 
Rapid-acting insulin analogues 
* Insulin Aspart 
* Insulin Glulisine 
* Insulin Lispro 
Short-acting (Regular) insulins 
Basal insulins 
Intermediate-acting (NPH) 
Long-acting insulin analogues 
* Insulin Detemir 
* Insulin Glargine 
* Insulin Degludec 
Premix insulins 
* Biphasic Human Insulin 
* Biphasic Insulin Aspart 
* Insulin Lispro + Insulin Lispro 
Protamine 
Coformulation 
* Insulin Degludec + 
Insulin Aspart   
of action 
10-15 min 
10-15 min 
10-15 min 
30 min 
1-3 hours 
90 min 
90 min 
30 mins 
10-20 mins 
10-20 mins 
30-90 mins 
10-20 min 
NB. Inhaled rapid acting insulin is also available 
Insulin regimens 
  
1-1.5 hour 
1-1.5 hour 
1-2 hour 
2-3 hours 
5-8 hours 
N/A 
N/A 
2-8 hours 
1-4 hours 
1-4 hours 
NA 
40-90 min   
of action 
3-5 hours 
3-5 hours 
3.5-4.75 hours 
6.5 hours 
Upto 18 hours 


In [96]:
cluster_chunker = ClusterSemanticChunker(
    embedding_function=embedding_function, 
    max_chunk_size=100, 
    length_function=token_count
)

cluster_chunker_chunks = cluster_chunker.split_text(all_text)

analyze_chunks(cluster_chunker_chunks, use_tokens=True)


Number of Chunks: 451

 ================================================== 200th Chunk ================================================== 
 Anion gap >10 >12 >12 Variable 
Mental status Alert Alert/ drowsy Stupor/coma Stupor/coma

 ================================================== 201st Chunk ================================================== 
 NB: Effective serum osmolality: 2 [measured Nat (mEq/L) + Glucose (mmol/L); Anion Gap: (Na*) 
-[Cl + HCO3 (mEq/L)]

No token overlap found


# LLM Semantic Chunker

In [101]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained model for semantic text similarity
model = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L6-v2")

# Function to split the document into sentences or parts
def split_document_into_sentences(all_text):
    # Splitting text into sentences (you could use other text-splitting strategies)
    sentences = all_text.split(". ")
    return sentences

# Function to perform semantic chunking
def semantic_chunking(all_text, chunk_size=3, similarity_threshold=0.8):
    sentences = split_document_into_sentences(all_text)

    # Get embeddings for all sentences in the document
    sentence_embeddings = model.encode(sentences)
    
    # Initialize a list to store chunks
    chunks = []
    current_chunk = []

    for i, embedding in enumerate(sentence_embeddings):
        if not current_chunk:
            current_chunk.append(sentences[i])
        else:
            # Calculate the similarity between the last sentence of the current chunk and the next sentence
            last_embedding = model.encode([current_chunk[-1]])  # Embedding of the last sentence in the chunk
            similarity = np.dot(embedding, last_embedding[0]) / (np.linalg.norm(embedding) * np.linalg.norm(last_embedding[0]))

            # If the similarity is above a threshold, add it to the current chunk
            if similarity > similarity_threshold:
                current_chunk.append(sentences[i])
            else:
                # Start a new chunk
                chunks.append(". ".join(current_chunk))
                current_chunk = [sentences[i]]

    # Add the last chunk
    if current_chunk:
        chunks.append(". ".join(current_chunk))

    return chunks



# Perform semantic chunking
chunks = semantic_chunking(all_text)

# Output the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk}\n")


Chunk 1: DIABETES CARE 
BADAS Guideline 2019 
          
  
   
  
   P|) 
DAS GUELINE ON Man 
DELIT IGEMEN 
  
A Joint Initiative of 
Diabetic Association of Bangladesh 
NCDC Program, Directorate General of Health Services DIABETES CARE 
BADAS Guideline 2019 
  
A Joint Initiative of 
Diabetic Association of Bangladesh 
NCDC Program, Directorate General of Health Services 
  
Diabetes Care: BADAS Guideline 2019 HEI! 1 DIABETES CARE: BADAS GUIDELINE 2019 
Convener: Prof A K Azad Khan 
Chairman: Prof Hajera Mahtab 
Members of the steering committee 
Prof Dr AHM Enayet Hossain 
Prof Akhtar Hussain 
Prof Zafar Anmed Latif 
Prof Tofail Ahmed 
Prof Laique Ahmed Khan 
Prof Nazrul Islam Siddiqui 
Prof Md Hafizur Rahman 
Prof Abdus Saleque Mollah 
Prof Md Farid Uddin 
Prof M A Jalil Ansary 
Prof Dr MA Samad 
Prof SM Ashrafuzzaman 
Prof MA Hasnat 
Dr Kazi Ali Hassan 
Dr Abdul Mannan Sarker 
Members of the Task Force 
Prof Md Faruque Pathan 
Dr Tareen Ahmed 
Dr Md Feroz Amin 
Dr Faria Afsana 
Dr

In [102]:
print("First chunk:", chunks[0])
print("Second chunk:", chunks[1])

# Get the number of chunks
num_chunks = len(chunks)
print(f"Total number of chunks: {num_chunks}")

First chunk: DIABETES CARE 
BADAS Guideline 2019 
          
  
   
  
   P|) 
DAS GUELINE ON Man 
DELIT IGEMEN 
  
A Joint Initiative of 
Diabetic Association of Bangladesh 
NCDC Program, Directorate General of Health Services DIABETES CARE 
BADAS Guideline 2019 
  
A Joint Initiative of 
Diabetic Association of Bangladesh 
NCDC Program, Directorate General of Health Services 
  
Diabetes Care: BADAS Guideline 2019 HEI! 1 DIABETES CARE: BADAS GUIDELINE 2019 
Convener: Prof A K Azad Khan 
Chairman: Prof Hajera Mahtab 
Members of the steering committee 
Prof Dr AHM Enayet Hossain 
Prof Akhtar Hussain 
Prof Zafar Anmed Latif 
Prof Tofail Ahmed 
Prof Laique Ahmed Khan 
Prof Nazrul Islam Siddiqui 
Prof Md Hafizur Rahman 
Prof Abdus Saleque Mollah 
Prof Md Farid Uddin 
Prof M A Jalil Ansary 
Prof Dr MA Samad 
Prof SM Ashrafuzzaman 
Prof MA Hasnat 
Dr Kazi Ali Hassan 
Dr Abdul Mannan Sarker 
Members of the Task Force 
Prof Md Faruque Pathan 
Dr Tareen Ahmed 
Dr Md Feroz Amin 
Dr Faria Afsana